# [모듈 2.0] 모델 빌딩하기 (No VPC 환경에서 실행하세요)



### Tensorflow 2.0 버전 이상 확인
이 노트북은 tensorflow 2.0 이상을 사용 합니다. 커널은 **"Conda_tensorflow2_p36"** 혹은 아래 pip로 설치해서 하시면 됩니다. 

In [1]:
# !pip install tensorflow==2.0.0

In [3]:
import tensorflow as tf
print(tf.__version__)

2.1.2


### Test your script locally (just like on your laptop)

테스트를 위해 위와 동일한 명령(command)으로 새 스크립트를 실행하고, 예상대로 실행되는지 확인합니다. <br>
SageMaker TensorFlow API 호출 시에 환경 변수들은 자동으로 넘겨기지만, 로컬 주피터 노트북에서 테스트 시에는 수동으로 환경 변수들을 지정해야 합니다. (아래 예제 코드를 참조해 주세요.)

```python
%env SM_MODEL_DIR=./logs
```

# (3) Use SageMaker local for local testing

본격적으로 학습을 시작하기 전에 로컬 모드를 사용하여 디버깅을 먼저 수행합니다. 로컬 모드는 학습 인스턴스를 생성하는 과정이 없이 로컬 인스턴스로 컨테이너를 가져온 후 곧바로 학습을 수행하기 때문에 코드를 보다 신속히 검증할 수 있습니다.

Amazon SageMaker Python SDK의 로컬 모드는 TensorFlow 또는 MXNet estimator서 단일 인자값을 변경하여 CPU (단일 및 다중 인스턴스) 및 GPU (단일 인스턴스) SageMaker 학습 작업을 에뮬레이션(enumlate)할 수 있습니다. 이를 위해 Docker compose와 NVIDIA Docker를 사용합니다.
학습 작업을 시작하기 위해 `estimator.fit() ` 호출 시, Amazon ECS에서 Amazon SageMaker TensorFlow 컨테이너를 로컬 노트북 인스턴스로 다운로드합니다.

로컬 모드의 학습을 통해 여러분의 코드가 현재 사용 중인 하드웨어를 적절히 활용하고 있는지 확인하기 위한 GPU 점유와 같은 지표(metric)를 쉽게 모니터링할 수 있습니다.

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

`sagemaker.tensorflow` 클래스를 사용하여 SageMaker Python SDK의 Tensorflow Estimator 인스턴스를 생성합니다.
인자값으로 하이퍼파라메터와 다양한 설정들을 변경할 수 있습니다.

자세한 내용은 [documentation](https://sagemaker.readthedocs.io/en/stable/using_tf.html#training-with-tensorflow-estimator)을 확인하시기 바랍니다.

In [2]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'loss: (.*?) '},
    {'Name': 'train:accuracy', 'Regex': 'acc: (.*?) '},
    {'Name': 'validation:loss', 'Regex': 'val_loss: (.*?) '},
    {'Name': 'validation:accuracy', 'Regex': 'val_acc: (.*?) '}
]

In [3]:
# %%time
# from sagemaker.tensorflow import TensorFlow
# estimator = TensorFlow(base_job_name='cifar10-pipe',
#                        entry_point='cifar10_keras_sm_pipe_tf2.py',
#                        source_dir='training_script',
#                        role=role,
#                        framework_version='2.0.0',
#                        py_version='py3',
#                        script_mode=True,
#                        hyperparameters={'epochs' : 1},
#                        train_instance_count=1, 
#                        train_instance_type='local',
#                        metric_definitions=metric_definitions, # 1_Monitoring_your_TensorFlow_scripts.ipynb 참조                       
#                        input_mode='Pipe' # 추가                       
#                       )


# estimator.fit({'train': 'file://data/train',
#                'validation': 'file://data/validation',
#                'eval': 'file://data/eval'})

학습을 수행할 3개의 채널과 데이터의 경로를 지정합니다. **로컬 모드로 수행하기 때문에 S3 경로 대신 노트북 인스턴스의 경로를 지정하시면 됩니다.**

Estimator가 처음 실행될 때 Amazon ECR 리포지토리(repository)에서 컨테이너 이미지를 다운로드해야 하지만 학습을 즉시 시작할 수 있습니다. 즉, 별도의 학습 클러스터가 프로비저닝 될 때까지 기다릴 필요가 없습니다. 또한 반복 및 테스트시 필요할 수 있는 후속 실행에서 MXNet 또는 TensorFlow 스크립트에 대한 수정 사항이 즉시 실행되기 시작합니다.

# (4) Using SageMaker for faster training time

이번에는 로컬 모드를 사용하지 않고 SageMaker 학습에 GPU 학습 인스턴스를 생성하여 학습 시간을 단축해 봅니다.<br>
로컬 모드와 다른 점들은 (1) `train_instance_type`이 로컬 모드의 ‘local’ 대신 여러분이 원하는 특정 인스턴스 유형으로 설정해야 하고, (2) 학습 데이터를 Amazon S3에 업로드 후 학습 경로를 S3 경로로 설정해야 합니다. 

SageMaker SDK는 S3 업로드를 위한 간단한 함수(`Session.upload_data()`)를 제공합니다. 이 함수를 통해 리턴되는 값은 데이터가 저장된 S3 경로입니다.
좀 더 자세한 설정이 필요하다면 SageMaker SDK 대신 boto3를 사용하시면 됩니다.

*[Note]: 고성능 워크로드를 위해 Amazon EFS와 Amazon FSx for Lustre도 지원하고 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/*

In [11]:
dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-ap-northeast-2-057716757052/data/DEMO-cifar10'

S3에 데이터 업로드를 완료했다면, Estimator를 새로 생성합니다. <br>
아래 코드를 그대로 복사 후에 `train_instance_type='local'`을 `train_instance_type='ml.p2.xlarge'`로 수정하고
`hyperparameters={'epochs': 1}`를 `hyperparameters={'epochs': 5}`로 수정합니다.

```python
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 1},
                       train_instance_count=1, 
                       train_instance_type='local')
```

*[Note] 
2019년 8월부터 SageMaker에서도 학습 인스턴스에 EC2 spot instance를 사용하여 비용을 크게 절감할 수 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/ko/blogs/korea/managed-spot-training-save-up-to-90-on-your-amazon-sagemaker-training-jobs/*

만약 Managed Spot Instance로 학습하려면 다음 코드를 Estimator의 train_instance_type의 다음 행에 추가해 주세요.
```python
train_max_run = 3600,
train_use_spot_instances = 'True',
train_max_wait = 3600,
```

In [12]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10-pipe',
                       entry_point='cifar10_keras_sm_pipe_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1, 
                       train_instance_type='ml.p3.8xlarge',
                       metric_definitions=metric_definitions, # 1_Monitoring_your_TensorFlow_scripts.ipynb 참조                       
                       input_mode='Pipe' # 추가                       
                      )



train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


학습을 수행합니다. 이번에는 각각의 채널(`train, validation, eval`)에 S3의 데이터 저장 위치를 지정합니다.<br>
학습 완료 후 Billable seconds도 확인해 보세요. Billable seconds는 실제로 학습 수행 시 과금되는 시간입니다.
```
Billable seconds: <time>
```

참고로, `ml.p2.xlarge` 인스턴스로 5 epoch 학습 시 전체 6분-7분이 소요되고, 실제 학습에 소요되는 시간은 3분-4분이 소요됩니다.

In [13]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2021-02-23 03:55:28 Starting - Starting the training job...
2021-02-23 03:55:51 Starting - Launching requested ML instancesProfilerReport-1614052527: InProgress
.........
2021-02-23 03:57:12 Starting - Preparing the instances for training...
2021-02-23 03:57:52 Downloading - Downloading input data
2021-02-23 03:57:52 Training - Downloading the training image......
2021-02-23 03:58:53 Training - Training image download completed. Training in progress.2021-02-23 03:58:40,387 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-02-23 03:58:40,875 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
   

In [24]:
%store dataset_location

Stored 'dataset_location' (str)
